<a href="https://colab.research.google.com/github/anadiedrichs/2021-tesis-experimentos/blob/main/tutorials/keras_sample_weight/regresion_lineal_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión lineal en keras
## Librerias

In [ ]:
install.packages("keras")

In [ ]:
library(keras)

In [ ]:
library(tidyverse)

## Primer ejemplo

Tomado de https://www.datatechnotes.com/2019/01/regression-example-with-keras-in-r.html

### Dataset 

In [ ]:
set.seed(123)
N = 450
n = seq(1:N)
a = n/10+4*sin(n/10)+sample(-1:6,N,replace=T)+rnorm(N)
b = n/8+4*sin(n/10)+sample(-3:3,N,replace=T)+rnorm(N) 
c = n/6+4*sin(n/10)+sample(-5:1,N,replace=T)+rnorm(N)
y = (a+b+c)/3+rnorm(N)
 
plot(n, c, col="orange", pch=20, cex=.9)
points(n, a, col="blue", pch=20, cex=.9)
points(n, b, col="green", pch=20, cex=.9)
points(n, y, col="red", type = "l",lwd=2)

In [ ]:
x = as.matrix(data.frame(a,b,c))
y = as.matrix(y)
head(x)

In [ ]:
model = keras_model_sequential() %>% 
   layer_dense(units=64, activation="relu", input_shape=3) %>% 
   layer_dense(units=32, activation = "relu") %>% 
   layer_dense(units=1, activation="linear")
 
model %>% compile(
   loss = "mse",
   optimizer =  "adam", 
   metrics = list("mean_absolute_error")
 )
 
model %>% summary()

In [ ]:
model %>% fit(x, y, epochs = 100,verbose = 0)
 
scores = model %>% evaluate(x, y, verbose = 0)
print(scores)

In [ ]:
y_pred = model %>% predict(x)

x_axes = seq(1:length(y_pred))
plot(x_axes, y, type="l", col="red")
lines(x_axes, y_pred, col="blue")
legend("topleft", legend=c("y-original", "y-predicted"),
        col=c("red", "blue"), lty=1,cex=0.8)

Ahora evaluamos la predicción en el conjunto de testeo

In [ ]:
train_x = x[1:400,]
test_x = x[401:N,]
 
train_y = y[1:400]
test_y = y[401:N]
 
model %>% fit(train_x,train_y,epochs = 100,verbose = 0,sample_weight = rep(1,400),)
y_pred = model %>% predict(test_x)

In [ ]:
help(rep)

In [ ]:
x_axes = seq(1:length(y_pred))
plot(x_axes, test_y, col="red", type="l")
lines(x_axes, y_pred, col="blue")
legend("topleft", legend=c("y-original", "y-predicted"),
        col=c("red", "blue"), lty=1,cex=0.8)

## Segundo ejemplo

tomado de https://www.petervanney.com/2019/08/17/r-tensorflow-and-keras-regression/


### librerias

In [ ]:
install.packages("recipes")

In [ ]:
install.packages("rsample")

In [ ]:
install.packages("yardstick")

In [ ]:
library(tidyverse)
library(lubridate)
library(tensorflow)
library(keras)
library(recipes)
library(rsample)
library(yardstick)
options(yardstick.event_first = FALSE)

In [ ]:
reticulate::py_config()

### dataset 

In [ ]:
dataset_path = get_file("auto-mpg.data",
                        "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")

In [ ]:
dataset_path

In [ ]:
column_names = c('MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin')

In [ ]:
auto_mpg = read_delim(file = dataset_path, 
                      col_names = column_names, 
                      delim = " ", 
                      comment = "\t", 
                      trim_ws = TRUE,
                      na = "?")

In [ ]:
head(auto_mpg)

In [ ]:
dataset = auto_mpg %>% 
  drop_na() %>% 
  mutate("USA" = if_else(Origin == 1,1,0),
         "Europe" = if_else(Origin == 2,1,0),
         "Japan" = if_else(Origin == 3,1,0)) %>% 
  select(-Origin)


### Training y testing sets

In [ ]:
data_split <- initial_split(dataset, prop = 4/5, strata = NULL)
training_data <- training(data_split)
testing_data <- testing(data_split)

examine training data 

In [ ]:
panel.hist <- function(x, ...)
{
  usr <- par("usr"); on.exit(par(usr))
  par(usr = c(usr[1:2], 0, 1.5) )
  h <- hist(x, plot = FALSE)
  breaks <- h$breaks; nB <- length(breaks)
  y <- h$counts; y <- y/max(y)
  rect(breaks[-nB], 0, breaks[-1], y, col = "cyan", ...)
}

pairs(training_data %>% 
        select(-USA, -Europe, -Japan), 
      diag.panel = panel.hist)

In [ ]:
help(hist)

In [ ]:
t(summary(training_data, digits = 2))